In [2]:
%matplotlib inline

import numpy as np
import collections
from collections import OrderedDict, Counter, defaultdict
import pandas as pd

import Bio
from Bio import SeqIO
from Bio.Seq import Seq

import seaborn as sns
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_file, show 
import matplotlib.patches as patches    
from matplotlib import colors


import glob

import subprocess
from subprocess import call

import re

import pickle

from Bio.SubsMat import MatrixInfo
import Bio.Data.CodonTable

from scipy import stats


from Bio.SubsMat import MatrixInfo
import Bio.Data.CodonTable

import joypy

import pyBigWig

In [3]:
path = '/home/alla/make_PhD_great_again/NTE/VERSION_since_May2021/'

# gencode 25
metadata_pc_g25 = pd.read_csv(path+'tmp_res/metadata_pc_g25.txt', sep='\t')
metadata_pc_g25['tr_id1'] = [x.split('.')[0] for x in metadata_pc_g25['tr_id'].tolist()] 

# gencode 41
metadata_pc_g41 = pd.read_csv('tmp_res/metadata_pc_g41.txt', sep='\t')
metadata_pc_g41['tr_id1'] = [x.split('.')[0] for x in metadata_pc_g41['tr_id'].tolist()] 
metadata_pc_g41[0:2]


# open metadata with scores and metrics
meta = pd.read_csv('/home/alla/make_PhD_great_again/NTE/VERSION_since_May2021/tmp_res/METADATA_CURRENT.txt', sep='\t')
meta[0:2]

,tr_id,N_term_start,N_term_end,len,N_term_start1,N_term_end1,len_codons,gene,50len_flag,global_coo_50_and_less,...,gene_ext,Start_ext,Stop_ext,global_rank_ext,Start_codon_ext,Proteomics_count_ext,Coverage_value_ext,CDS_ratio,Riboseq_Summary,Nstart_Nend
0,ENST00000624431.1,274,316,42,274,316,14.0,FO538757.2,274,chr1:182667-182708,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,274-316
1,ENST00000624735.1,0,236,236,2,236,78.0,FO538757.1,86,chr1:200087-200236,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2-236


# nonAUG TIS 

In [5]:
TIS_efficiency_df = pd.read_csv('/home/alla/make_PhD_great_again/NTE/VERSION_since_May2021/data/TIS_nonAUG_efficiency.txt', sep='\t')

TIS_efficiency_df['scaled_efficiency'] = TIS_efficiency_df['TIS Efficiency'] / TIS_efficiency_df['TIS Efficiency'].max()

TIS_efficiency_dict = dict(zip(TIS_efficiency_df['TIS Sequence'], TIS_efficiency_df['TIS Efficiency']))

TIS_efficiency_dict_scaled = dict(zip(TIS_efficiency_df['TIS Sequence'], TIS_efficiency_df['scaled_efficiency']))

TIS_efficiency_df.sort_values(by='TIS Efficiency', ascending=False).head(4)

,TIS Sequence,TIS Efficiency,scaled_efficiency
5977,CGCCCUGG,50.4,1.000000
6041,CGCGCUGG,50.4,1.000000
5913,CGCACUGG,50.4,1.000000
6001,CAUCCUGG,47.6,0.944444


# AUG TIS 

In [6]:
TIS_AUG_eff = pd.read_csv('/home/alla/make_PhD_great_again/NTE/VERSION_since_May2021/data/TIS_AUG_efficiency.txt', sep='\t')

TIS_AUG_eff['scaled_efficiency'] = TIS_AUG_eff['efficiency'] / TIS_AUG_eff['efficiency'].max()

print (TIS_AUG_eff.efficiency.max(), TIS_AUG_eff.efficiency.min())


TIS_AUG_eff_dict = dict(zip(TIS_AUG_eff['sequence'], TIS_AUG_eff['efficiency']))


TIS_AUG_eff_dict_scaled = dict(zip(TIS_AUG_eff['sequence'], TIS_AUG_eff['scaled_efficiency']))
                           
TIS_AUG_eff[0:5]

150 12


,sequence,efficiency,lower.bound,upper.bound,scaled_efficiency
0,UUCAUCAUGCA,150,139,161,1.000000
1,GCGAGUAUGGC,149,139,161,0.993333
2,ACGAGAAUGGC,148,136,160,0.986667
3,AUCAUCAUGCA,147,136,158,0.980000
4,ACGAGUAUGGC,145,134,157,0.966667


In [9]:
li = []

gene = 'CCDC8'

tr_id = 'ENST00000307522'

pre_st = '565_CTG'

li.append([gene, tr_id, int(pre_st.split('_')[0]), pre_st.split('_')[1]])

gene = 'EIF4G2'

tr_id = 'ENST00000339995'

pre_st = '493_GTG'

li.append([gene, tr_id, int(pre_st.split('_')[0]), pre_st.split('_')[1]])


gene = 'BAG1'

tr_id = 'ENST00000472232'

pre_st = '88_CTG'

li.append([gene, tr_id, int(pre_st.split('_')[0]), pre_st.split('_')[1]])


gene = 'GTF3A'

tr_id = 'ENST00000381140'

pre_st = '195_CTG'

li.append([gene, tr_id, int(pre_st.split('_')[0]), pre_st.split('_')[1]])


gene = 'DDX17'

tr_id = 'ENST00000396821'

pre_st = '101_CTG'

li.append([gene, tr_id, int(pre_st.split('_')[0]), pre_st.split('_')[1]])


gene = 'RNF187'

tr_id = 'ENST00000305943'

pre_st = '429_CTG'

li.append([gene, tr_id, int(pre_st.split('_')[0]), pre_st.split('_')[1]])



gene = 'NELFB'

tr_id = 'ENST00000343053'

pre_st = '194_CTG'

li.append([gene, tr_id, int(pre_st.split('_')[0]), pre_st.split('_')[1]])



gene = 'NPTXR'

tr_id = 'ENST00000333039'

pre_st = '125_CTG'

li.append([gene, tr_id, int(pre_st.split('_')[0]), pre_st.split('_')[1]])



gene = 'NPW'

tr_id = 'ENST00000329610'

pre_st = '383_CTG'

li.append([gene, tr_id, int(pre_st.split('_')[0]), pre_st.split('_')[1]])



gene = 'PIGX'

tr_id = 'ENST00000392391'

pre_st = '176_CTG'

li.append([gene, tr_id, int(pre_st.split('_')[0]), pre_st.split('_')[1]])




gene = 'PRPS1L1'

tr_id = 'ENST00000506618'

pre_st = '82_ACG'

li.append([gene, tr_id, int(pre_st.split('_')[0]), pre_st.split('_')[1]])



#gene = 'PRR16'

#tr_id = 'ENST00000446965'

#pre_st = '82_GTG'

#li.append([gene, tr_id, int(pre_st.split('_')[0]), pre_st.split('_')[1]])




gene = 'SLC22A17'

tr_id = 'ENST00000397260'

pre_st = '120_CTG'

li.append([gene, tr_id, int(pre_st.split('_')[0]), pre_st.split('_')[1]])



gene = 'SP3'

tr_id = 'ENST00000418194'

pre_st = '385_ATA'

li.append([gene, tr_id, int(pre_st.split('_')[0]), pre_st.split('_')[1]])




gene = 'SWI5'

tr_id = 'ENST00000608796'

pre_st = '120_CTG'

li.append([gene, tr_id, int(pre_st.split('_')[0]), pre_st.split('_')[1]])



gene = 'TXNRD3'

tr_id = 'ENST00000524230'

pre_st = '166_CTG'

li.append([gene, tr_id, int(pre_st.split('_')[0]), pre_st.split('_')[1]])


gene = 'R3HCC1'

tr_id = 'ENST00000265806'

pre_st = '269_CTG'

li.append([gene, tr_id, int(pre_st.split('_')[0]), pre_st.split('_')[1]])

df = pd.DataFrame(li, columns = ['gene', 'tr_id1', 'start', 'codon'])

In [13]:
metadata_pc_g25['tr_id1'] = [x.split('.')[0] for x in metadata_pc_g25['tr_id'].tolist()]

In [19]:
df[0:2]

,gene,tr_id1,start,codon
0,CCDC8,ENST00000307522,565,CTG
1,EIF4G2,ENST00000339995,493,GTG


In [23]:
li = []
for el in df.to_numpy():
    seq = metadata_pc_g25[metadata_pc_g25['tr_id1'] == el[1]].iloc[0].transcript_seq
    start = el[2]
    nonAUG_context = seq[start-5:start+6]
    li.append([el[0], el[1], el[2], el[3], nonAUG_context])
    
    # nonAUG TIS score 

pd.DataFrame(li)

,0,1,2,3,4
0,CCDC8,ENST00000307522,565,CTG,CACCCTGGAGA
1,EIF4G2,ENST00000339995,493,GTG,CAAAGTGGAGA
2,BAG1,ENST00000472232,88,CTG,GGGCCTGGCTC
3,GTF3A,ENST00000381140,195,CTG,CGCCCTGGATC
4,DDX17,ENST00000396821,101,CTG,AAACCTGCCCA
5,RNF187,ENST00000305943,429,CTG,AGCCCTGGCGC
6,NELFB,ENST00000343053,194,CTG,GGACCTGGCCG
7,NPTXR,ENST00000333039,125,CTG,CACGCTGAAGT
8,NPW,ENST00000329610,383,CTG,CGCCCTGGCGT
9,PIGX,ENST00000392391,176,CTG,CGTCCTGGCGG


In [10]:
df

,gene,tr_id1,start,codon
0,CCDC8,ENST00000307522,565,CTG
1,EIF4G2,ENST00000339995,493,GTG
2,BAG1,ENST00000472232,88,CTG
3,GTF3A,ENST00000381140,195,CTG
4,DDX17,ENST00000396821,101,CTG
5,RNF187,ENST00000305943,429,CTG
6,NELFB,ENST00000343053,194,CTG
7,NPTXR,ENST00000333039,125,CTG
8,NPW,ENST00000329610,383,CTG
9,PIGX,ENST00000392391,176,CTG


# TIS score of sink ORFs and nonAUG starts 

In [ ]:
could be actually like a two-sided barplot with showing top-score as dashed line for both types of codons 

# Any polyAla downstream of nonAUG starts? 

# Any quadruplexes? 